In [2]:
import pandas as pd
import seaborn as sns
sns.set() 
%matplotlib inline 
pd.set_option('max_columns',999)

In [ ]:
# PAYMENT
pay_data = pd.read_csv("data//PANEL//PAYMENT data.csv", engine = "python")
pay_brand = pd.read_csv("data//PANEL//PAYMENT Summary_Brand.csv", engine = "python")
pay_cat = pd.read_csv("data//PANEL//PAYMENT Summary_Brand.csv", engine = "python")
pay_total = pd.read_csv("data//PANEL//PAYMENT Summary_Total.csv", engine = "python")

In [ ]:
# PPDB
ppdb_data = pd.read_csv("data//PANEL//PPDB data.csv", engine = "python")

In [ ]:
# APP
app_time = pd.read_csv("data//PANEL//APP Refine_AppTime.csv", engine = "python")
app_cat_g = pd.read_csv("data//PANEL//APP Refine_CategoryTime_G.csv", engine = "python")
app_cat_mme = pd.read_csv("data//PANEL//APP Refine_CategoryTime_MME.csv", engine = "python")
app_sum_time = pd.read_csv("data//PANEL//APP Refine_Usage_Sum_Time.csv", engine = "python")

# Gephi


Gephi에 넣기 위해 다듬는 작업

## AR - Label

시각화 결과물에 label을 부여하기 위해 각 컬럼의 label을 지정




##### ps data 라벨링

In [45]:
# indiv(개인기본정보) / family(가족관련정보) / region(지역정보) 로 단순하게 구분!

PSdata_label = pd.DataFrame(data = ps_dummies.columns).rename(columns={0:'Colname'})[1:]
PSdata_label['Label'] = PSdata_label['Colname']\
    .map(lambda x: 'PS_family' if "family" in x 
                    else 'PS_region' if 'region' in x
                    else 'PS_indiv')

In [46]:
PSdata_label.head()

,Colname,Label
1,gender_1,PS_indiv
2,gender_2,PS_indiv
3,age_m_1,PS_indiv
4,age_m_2,PS_indiv
5,age_m_3,PS_indiv


In [47]:
PSdata_label.groupby('Label').count()

,Colname
Label,
PS_family,17
PS_indiv,32
PS_region,17


##### payment 데이터 라벨링



In [48]:
# indiv(개인기본정보) / family(가족관련정보) / region(지역정보) 로 단순하게 구분!

paydata_label = pd.DataFrame(data = pay_dummies.columns).rename(columns={0:'Colname'})[1:]
paydata_label['Label'] = paydata_label['Colname']\
    .map(lambda x: 'pay_category' if "CATEGORY" in x 
                    else 'pay_approval' if 'APPROVAL' in x
                    else 'pay_indiv')

In [49]:
paydata_label.head()

,Colname,Label
1,PAY_APPROVAL_TYPE_L,pay_approval
2,PAY_APPROVAL_TYPE_F,pay_approval
3,PAY_CATEGORY_CODE_S_1101,pay_category
4,PAY_CATEGORY_CODE_S_1799,pay_category
5,PAY_CATEGORY_CODE_S_1001,pay_category


In [50]:
paydata_label.groupby('Label').count()

,Colname
Label,
pay_approval,22
pay_category,63
pay_indiv,64


##### app 데이터 라벨링

In [51]:
# indiv(개인기본정보) / family(가족관련정보) / region(지역정보) 로 단순하게 구분!

appdata_label = pd.DataFrame(data = app_dummies.columns).rename(columns={0:'Colname'})[1:]
appdata_label['Label'] = appdata_label['Colname']\
    .map(lambda x: 'app_indiv')

In [52]:
appdata_label.groupby('Label').count()

,Colname
Label,
app_indiv,564


##### 전체 label 저장


In [150]:
allcol_label = pd.concat([PSdata_label, paydata_label, appdata_label])
allcol_label.columns = ['id','category']
category = allcol_label
category.to_csv("data/allcol_label.csv", index=False)

In [151]:
category.head(3)

,id,category
1,gender_1,PS_indiv
2,gender_2,PS_indiv
3,age_m_1,PS_indiv


### 원 크기 지정

In [152]:
size = pd.DataFrame(al.sum())
size = size.reset_index()
size.columns = ['id','size']
size.head()

,id,size
0,PAY_APPROVAL_TYPE_L,10586
1,PAY_APPROVAL_TYPE_F,1009
2,PAY_CATEGORY_CODE_S_1101,3981
3,PAY_CATEGORY_CODE_S_1799,10014
4,PAY_CATEGORY_CODE_S_1001,2949


In [153]:
# size 1에서 10사이로 scaling
size["size"] = size["size"] // 1000
size["size"] = [x+1 for x in size["size"]]
size["size"] = [10 if x == 11 else x for x in size["size"]]

In [154]:
size.to_csv("data/size_label.csv", index = False)

In [55]:
rules_for_network = rules_for_network.reset_index(drop=True)
rules_for_network.columns = ['Source','Target','Weight']

In [56]:
rules_for_network.head()

,Source,Target,Weight
0,app_name_NotiScreen,app_name_NotiToMe,24.257902
1,app_name_공영쇼핑,app_name_아임쇼핑,20.919363
2,app_name_썸뱅크,app_name_BNK통합인증,19.509737
3,education_1,age_m_1,18.521498
4,app_name_DGB 개인뱅킹,region_sido_3,17.031678


In [57]:
source_1 = rules_for_network['Source']
source_2 = rules_for_network['Target']
source = pd.concat([source_1,source_2]).drop_duplicates().reset_index(drop=True)
source = pd.DataFrame(source).rename(columns={0:'id'})

In [58]:
source.head()

,id
0,app_name_NotiScreen
1,app_name_공영쇼핑
2,app_name_썸뱅크
3,education_1
4,app_name_DGB 개인뱅킹


In [198]:
source[source["id"]=="app_name_미세미세"]

,id
222,app_name_미세미세


In [161]:
# source 개수에 맞춰서 category, size 개수 필터링
source_unique = list(source["id"].unique())

In [192]:
# 수정 category csv 만들기
category["bool"] = category.apply(lambda x : x["id"] in source_unique, axis = 1)
category_m = category[category["bool"] == True]
category_m = category_m.drop("bool", axis = 1)
category_m.to_csv("data/category_m.csv", index = False)

In [193]:
# 수정 size csv 만들기
size["bool"] = size.apply(lambda x : x["id"] in source_unique, axis = 1)
size_m = size[size["bool"] == True]
size_m = size_m.drop("bool", axis = 1)
size_m.to_csv("data/size_m.csv", index = False)